In [ ]:
import numpy as np
import pandas as pd
from dowhy import CausalModel
import graphviz
import warnings
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor
# EconML imports
from econml.dml import LinearDML, CausalForestDML
from econml.cate_interpreter import SingleTreeCateInterpreter, SingleTreePolicyInterpreter
import seaborn as sns
import dowhy.plotter
%matplotlib inline
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')



# The Direct Acyclic Graph

G=graphviz.Digraph()
G.edge("ta" , "tr")
G.edge("vel" , "ta")
G.edge("rh" , "ta")
G.edge("ta" , "clo")
G.edge("ta" , "thermal_sensation")
G.edge("vel" , "tr")
G.edge("rh" , "tr")
G.edge("tr" , "clo")
G.edge("tr" , "thermal_sensation")
G.edge("vel" , "clo")
G.edge("vel" , "thermal_sensation")
G.edge("rh" , "clo")
G.edge("rh" , "thermal_sensation")
G.edge("met" , "thermal_sensation")
G.edge("clo" , "thermal_sensation")

######################################
#########################################



G.edge("age" , "thermal_sensation")
G.edge("age" , "met")




# print(dot.source)

G.format = 'pdf'
G.render(directory='DAG', view = False).replace('\\', '/')



In [ ]:
# Reading the data
data = pd.read_excel("data.xlsx")



In [73]:
data_for_causal = data[["ta","tr", "vel", "rh", "met", "clo", "thermal_sensation", "age", "gender"]]
data_for_causal = data_for_causal.dropna()
data_for_causal = data_for_causal.loc[(data_for_causal['age'] >= 66)]

In [74]:
data_for_causal.groupby('gender').size()

gender
female    211
male      128
dtype: int64

In [80]:
age = data_for_causal.copy()
age = age.loc[age['gender'] == 'male']
age = age.drop(labels='gender', axis=1)

In [81]:
data_for_causal.shape, age.shape

In [82]:
# Initializing causal model
model = CausalModel(data=age,
                     graph=G.source.replace("\t", ' ').replace("\n", ' '),
                     treatment="age",
                     outcome="thermal_sensation")

# Identifying the estimation method
identified_estimand= model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                        
──────(E[thermalₛₑₙₛₐₜᵢₒₙ])
d[age]                     
Estimand assumption 1, Unconfoundedness: If U→{age} and U→thermal_sensation then P(thermal_sensation|age,,U) = P(thermal_sensation|age,)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!



In [83]:
random_state = 120
dml_estimate = model.estimate_effect(identified_estimand,
                                     method_name="backdoor.econml.dml.DML", # Calling EconMl double machine learning algorithm
                                     control_value = 0,
                                     treatment_value = 2,
                                     target_units = 'ate',
                                     confidence_intervals=False,
                                method_params={"init_params":{'model_y':GradientBoostingRegressor(random_state=random_state, learning_rate=0.0001),
                                                              'model_t': GradientBoostingRegressor(random_state=random_state, learning_rate=0.0001),
                                                              "model_final":LassoCV(fit_intercept=False, random_state=random_state),
                                                              'featurizer':PolynomialFeatures(degree=1, include_bias=True),
                                                              'random_state':random_state},
                                               "fit_params":{}})
print(dml_estimate.value)

-0.025927337047604193


In [84]:
# Random cause
res_random=model.refute_estimate(identified_estimand, dml_estimate, method_name="random_common_cause", random_seed=123)
print(res_random)

# Add Unobserved Common Causes
res_unobserved=model.refute_estimate(identified_estimand, dml_estimate, method_name="add_unobserved_common_cause",
                                     confounders_effect_on_treatment="linear", confounders_effect_on_outcome="linear",
                                    effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02, random_seed=123)
print(res_unobserved)

# Placebo Treatment
res_placebo=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="placebo_treatment_refuter", random_seed=123)
print(res_placebo)


# Data Subsets Validation
res_subset=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="data_subset_refuter", subset_fraction=0.8,
        num_simulations=10, random_seed=123)
print(res_subset)

Refute: Add a random common cause
Estimated effect:-0.025927337047604193
New effect:-0.02592626264269058
p value:0.44999999999999996

Refute: Add an Unobserved Common Cause
Estimated effect:-0.025927337047604193
New effect:-0.025891701729457928

Refute: Use a Placebo Treatment
Estimated effect:-0.025927337047604193
New effect:0.0
p value:1.0

Refute: Use a subset of data
Estimated effect:-0.025927337047604193
New effect:-0.028289326157824268
p value:0.3623471749051672

